In [277]:
#################
# LOAD MODULES
#################
import gmaps
import pandas as pd
import numpy as np
import datetime
import time
%matplotlib inline

In [307]:
# set current time
curr_time = datetime.datetime(2019,7,29,10,0,0)#datetime.datetime.now()
# set total number of data rows
total = 30
# declare random generator
rand = np.random

In [360]:
# declare dataframe headings for vehicle data
ev_data = pd.DataFrame(
    columns=['data_id','datetime','vrn_id','lat','long','current_kw','charge_freq','shift_id','events'])
# declare dataframe headings for depo data
ev_depo = pd.DataFrame(
    columns=['data_id','datetime','depo_id','total_CPs','used_CPs','max_kw/hr','current_kw/hr','events'])
# declare dataframe headings for cp data
ev_cp = pd.DataFrame(
    columns=['data_id','datetime','CP_id','CP_type','CP_status','CP_rate(kw/hr)','kw_left','hours_to_exit','hours_to_finish'])

# declare car parameters
current_kw = 30
charge_freq = 0
current_kw2 = 30
charge_freq2 = 0
# declare depo parameters
curr_charging = 0
used_cps = 0
# declare event parameters
last_event = -1
last_event2 = -1
events = {None:None, 0:"exit depo", 1:"enter depo", 2:"start charging", 3:"finish charging"}

for i in range(0,total):
    # each data differs by 5 minutes
    date_time = curr_time + datetime.timedelta(minutes=i*10)
    
    # realistically place events in dataframe
    if i == 0: eventtype = 0
    elif i == 12: eventtype = 1
    elif i == 14: eventtype = 2
    elif i == 26: eventtype = 3
    elif i == 28: eventtype = 0
    else: eventtype = None
        
    # for car3
    if i == 3: eventtype2 = 0
    elif i == 11: eventtype2 = 1
    elif i == 15: eventtype2 = 2
    elif i == 25: eventtype2 = 3
    elif i == 29: eventtype2 = 0
    else: eventtype2 = None
    
    # configure variables during events
    if eventtype is not None:
        last_event = eventtype
        if eventtype == 2:        # starts charging
            used_cps += 1             # increment number of available charge points by 1
            curr_charging += 7        # increment available resources for charging inside depo
        if eventtype == 3:        # finishes charging
            used_cps -= 1             # decrement number of available charge points by 1
            curr_charging -= 7        # decrement available resources for charging inside depo
            charge_freq += 1          # increment charge count by 1
    
    # configure variables between events
    if eventtype is None:
        if last_event == 0:                     # driving
            charge_freq += rand.choice([0,1,0])     # increment charge count by 0 or 1
            current_kw -= rand.uniform(1.5,3)       # simulate decrease in vehicle charge while driving
        if last_event == 2:                     # charging
            current_kw += 7/6                       # simulate charging at 7kw/hr (for 10 minute intervals)
            
    # configure variables during events
    if eventtype2 is not None:
        last_event2 = eventtype2
        if eventtype2 == 2:       # starts charging
            used_cps += 1             # increment number of available charge points by 1
            curr_charging += 7        # increment available resources for charging inside depo
        if eventtype2 == 3:       # finishes charging
            used_cps -= 1             # decrement number of available charge points by 1
            curr_charging -= 7        # decrement available resources for charging inside depo
            charge_freq2 += 1         # increment charge count by 1
    
    # configure variables between events
    if eventtype2 is None:
        if last_event2 == 0:                     # driving
            charge_freq2 += rand.choice([0,1,0])     # increment charge count by 0 or 1
            current_kw2 -= rand.uniform(1.5,3)       # simulate decrease in vehicle charge while driving
        if last_event2 == 2:                     # charging
            current_kw2 += 7/6                       # simulate charging at 7kw/hr (for 10 minute intervals)
        
    ##################
    # VEHICLE DATA
    ##################
    if i%2==0:
        ev_data = ev_data.append({
            'data_id': i,
            'datetime': date_time,
            'vrn_id': 7,                  # car id
            'lat': rand.uniform(45,60),   # random latitude within UK
            'long': rand.uniform(-2,2),   # random longitude within UK
            'current_kw': current_kw,     # current kw left in the vehicle
            'charge_freq': charge_freq,   # No. of times vehicle charges in a day
            'shift_id': 7,                # Predicted shift of vehicle
            'events': events[eventtype]
        }, ignore_index=True)
    else:
        ev_data = ev_data.append({
            'data_id': i,
            'datetime': date_time,
            'vrn_id': 3,                  # car id
            'lat': rand.uniform(45,60),   # random latitude within UK
            'long': rand.uniform(-2,2),   # random longitude within UK
            'current_kw': current_kw2,    # current kw left in the vehicle
            'charge_freq': charge_freq2,  # No. of times vehicle charges in a day
            'shift_id': 5,                # Predicted shift of vehicle
            'events': events[eventtype2]
        }, ignore_index=True)
    
    ##################
    # DEPO DATA
    ##################
    ev_depo = ev_depo.append({
        'data_id': i,
        'datetime': date_time,
        'depo_id': 1,                   # If >1 depo under management
        'total_CPs': 4,                 # total number of charging points
        'used_CPs': used_cps,           # number of used charging points
        'max_kw/hr': 14,                # maximum charging capacity of the whole depo
        'current_kw/hr': curr_charging, # current charging capacity of the whole depo
        'events': events[eventtype] if i%2==0 else events[eventtype2]
    }, ignore_index=True)
    
    ##################
    # CP DATA
    ##################
    # manually assign vehicle to charge point
    cp_id = rand.randint(0,4)
    if cp_id==3: cp_status = 7
    elif cp_id==2: cp_status = 3
    
    # calculate different parameters if charging point is occupied
    cp_rate = 7 if cp_status is not None else None
    charge_left = round(rand.uniform(0,30),3) if cp_status is not None else None
    hrs_to_finish = round(charge_left/cp_rate,3) if cp_status is not None else None
    hrs_to_exit = round(rand.uniform(0,20),3) if cp_status is not None else None
    
    ev_cp = ev_cp.append({
        'data_id': i,
        'datetime': date_time,
        'CP_id': cp_id,                   # charging point id (assume 4 CPs now)
        'CP_type': 1,                     # 0-slow, 1-fast (assume 2 types of CP for now)
        'CP_status': cp_status,           # shows vrn_id if occupied, else null (assume 4 cars for now)
        'CP_rate(kw/hr)': cp_rate,        # rate of charging, None if unoccupied (assume 7 kw/hr for now)
        'kw_left': charge_left,           # charge left to reach desired level, NaN if unoccupied (assume 30kw leaf)
        'hours_to_finish': hrs_to_finish, # hours needed to finish charging based on CP_rate, NaN if unoccupied
        'hours_to_exit': hrs_to_exit,     # hours till vehicle exits depo according to shift, NaN if unoccupied
    }, ignore_index=True)
    
# write to csv file
ev_data.to_csv('ev_data.csv')
ev_depo.to_csv('ev_depo.csv')
ev_cp.to_csv('ev_cp.csv')

In [361]:
ev_data#.iloc[::-1]

,data_id,datetime,vrn_id,lat,long,current_kw,charge_freq,shift_id,events
0,0,2019-07-29 10:00:00,7,50.154500,-1.217763,30,0,7,exit depo
1,1,2019-07-29 10:10:00,3,46.144801,0.335848,30,0,5,None
2,2,2019-07-29 10:20:00,7,56.657133,1.204102,25.8742,0,7,None
3,3,2019-07-29 10:30:00,3,59.034887,-0.922173,30,0,5,exit depo
4,4,2019-07-29 10:40:00,7,57.501458,-0.288556,21.7214,0,7,None
5,5,2019-07-29 10:50:00,3,55.340879,0.950601,25.7006,2,5,None
6,6,2019-07-29 11:00:00,7,47.257572,-1.269781,16.5221,0,7,None
7,7,2019-07-29 11:10:00,3,55.213825,0.197970,21.9216,2,5,None
8,8,2019-07-29 11:20:00,7,52.354151,1.741299,11.2001,1,7,None
9,9,2019-07-29 11:30:00,3,56.370793,1.602495,17.2179,3,5,None


In [362]:
ev_depo#.iloc[::-1]

,data_id,datetime,depo_id,total_CPs,used_CPs,max_kw/hr,current_kw/hr,events
0,0,2019-07-29 10:00:00,1,4,0,14,0,exit depo
1,1,2019-07-29 10:10:00,1,4,0,14,0,None
2,2,2019-07-29 10:20:00,1,4,0,14,0,None
3,3,2019-07-29 10:30:00,1,4,0,14,0,exit depo
4,4,2019-07-29 10:40:00,1,4,0,14,0,None
5,5,2019-07-29 10:50:00,1,4,0,14,0,None
6,6,2019-07-29 11:00:00,1,4,0,14,0,None
7,7,2019-07-29 11:10:00,1,4,0,14,0,None
8,8,2019-07-29 11:20:00,1,4,0,14,0,None
9,9,2019-07-29 11:30:00,1,4,0,14,0,None


In [363]:
ev_cp#.iloc[::-1]

,data_id,datetime,CP_id,CP_type,CP_status,CP_rate(kw/hr),kw_left,hours_to_exit,hours_to_finish
0,0,2019-07-29 10:00:00,1,1,7,7,22.661,18.078,3.237
1,1,2019-07-29 10:10:00,3,1,7,7,22.126,8.233,3.161
2,2,2019-07-29 10:20:00,3,1,7,7,3.029,10.473,0.433
3,3,2019-07-29 10:30:00,1,1,7,7,18.762,6.588,2.680
4,4,2019-07-29 10:40:00,1,1,7,7,1.620,6.785,0.231
5,5,2019-07-29 10:50:00,0,1,7,7,20.776,10.463,2.968
6,6,2019-07-29 11:00:00,1,1,7,7,29.059,10.226,4.151
7,7,2019-07-29 11:10:00,0,1,7,7,29.357,13.679,4.194
8,8,2019-07-29 11:20:00,0,1,7,7,15.549,7.705,2.221
9,9,2019-07-29 11:30:00,1,1,7,7,20.283,19.550,2.898
